In [1]:
import numpy as np

### Step1. RNNユニットを作成

In [2]:
N = 32
T = 16
D = 100
H = 100

# バッチサイズ 32, タイムステップ 16, 次元数 100 のテンソルを生成
xs = np.random.randn(N, T, D)

# 1 時刻分を取り出す
x = xs[:, 0, :]
x.shape

(32, 100)

$$ tanh(h_{t-1}W_{h} + x_{t}W_{x} + b) = h_{t} $$
$$ h_{t-1} : (N, H) $$
$$  W_{h} : (H, H) $$
$$ x_{t} : (N, D) $$
$$ W_{x} : (D, H) $$
$$ b : (N, H) $$
$$ h_{t} : (N, H) $$

In [3]:
# 隠れ状態ベクトルの初期値は 0 とする
h_prev = np.zeros((N, H))

# パラメータ Wh, Wx, b を生成
Wh = np.random.randn(H, H)
Wx = np.random.randn(D, H)
b = np.random.randn(N, H)

In [4]:
u = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
u.shape

(32, 100)

In [5]:
h_next = np.tanh(u)
h_next.shape

(32, 100)

In [6]:
h_next

array([[ 0.9881369 ,  0.9999999 ,  0.99955073, ...,  0.99999999,
         0.99887534,  0.98743934],
       [ 1.        , -0.99999711, -0.95491545, ...,  0.99999999,
        -0.99990978,  0.9998628 ],
       [ 0.99999897, -0.99998094,  1.        , ...,  0.99843245,
        -1.        ,  1.        ],
       ...,
       [-1.        ,  1.        ,  0.99996976, ...,  1.        ,
         1.        , -1.        ],
       [-0.77173216, -0.09944896,  0.99999455, ...,  0.12510887,
        -1.        ,  0.81339823],
       [-1.        ,  0.999996  ,  0.99999979, ..., -1.        ,
         0.99999131,  0.99239536]])

In [7]:
class RNN:
  def __init__(self, Wx, Wh, b): # 引数として、 2 つの重みと 1 つのバイアスをとる
    self.params = [Wx, Wh, b] # パラメータを格納
  
  def forward(self, x, h_prev):
    Wx, Wh, b = self.params
    t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b # 線形変換
    h_next = np.tanh(t) # 非線形変換

    self.cache = (x, h_prev, h_next)
    return h_next

In [8]:
rnn_unit = RNN(Wx, Wh, b)
res = rnn_unit.forward(x, h_prev)

In [9]:
res == h_next

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

### Step2. RNNレイヤーを作成

In [10]:
xs = np.random.randn(N, T, D)

In [11]:
rnn_unit = RNN(Wx, Wh, b)

In [12]:
h_0 = np.zeros((N, H))
h_1 = rnn_unit.forward(xs[:, 0, :], h_0)
h_2 = rnn_unit.forward(xs[:, 1, :], h_1)
h_3 = rnn_unit.forward(xs[:, 2, :], h_2)

In [13]:
N, T, D = xs.shape

In [14]:
h = np.zeros((N, H), dtype='f') # 隠れ状態ベクトルの初期値
hs = np.empty((N, T, H), dtype='f')
for t in range(T):
  # RNN セルで隠れ状態ベクトルを更新
  h = rnn_unit.forward(xs[:, t, :], h)
  hs[:, t, :] = h

In [15]:
hs.shape

(32, 16, 100)

In [16]:
class RNNLayer:
  def __init__(self, Wx, Wh, b, stateful=False):
    self.params = [Wx, Wh, b]
    self.rnn = RNN(Wx, Wh, b)
    self.h = None
    self.stateful = stateful

  def forward(self, xs):
    Wx, Wh, b = self.params 
    N, T, D = xs.shape 
    D, H = Wx.shape 

    hs = np.empty((N, T, H), dtype='f')
    
    if not self.stateful or self.h is None:
      self.h = np.zeros((N, H), dtype='f')

    for t in range(T):
      self.h = self.rnn.forward(xs[:, t, :], self.h)
      hs[:, t, :] = self.h 

    return hs

In [17]:
rnn_layer = RNNLayer(Wx, Wh, b, stateful=False)

In [18]:
hs = rnn_layer.forward(xs)

In [19]:
hs.shape

(32, 16, 100)